# Введение в вероятность и статистику
В этой тетради мы поиграем с некоторыми концепциями, которые мы обсуждали ранее. Многие концепции из области вероятности и статистики хорошо представлены в основных библиотеках для обработки данных на Python, таких как `numpy` и `pandas`.


In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

## Случайные величины и распределения
Давайте начнем с выборки из 30 значений из равномерного распределения от 0 до 9. Мы также вычислим среднее и дисперсию.


In [ ]:
sample = [ random.randint(0,10) for _ in range(30) ]
print(f"Sample: {sample}")
print(f"Mean = {np.mean(sample)}")
print(f"Variance = {np.var(sample)}")

Чтобы визуально оценить, сколько различных значений содержится в выборке, мы можем построить **гистограмму**:


In [ ]:
plt.hist(sample)
plt.show()

## Анализ реальных данных

Среднее значение и дисперсия очень важны при анализе данных из реального мира. Давайте загрузим данные о бейсболистах с сайта [SOCR MLB Height/Weight Data](http://wiki.stat.ucla.edu/socr/index.php/SOCR_Data_MLB_HeightsWeights)


In [ ]:
df = pd.read_csv("../../data/SOCR_MLB.tsv",sep='\t', header=None, names=['Name','Team','Role','Weight','Height','Age'])
df


> Здесь мы используем пакет под названием [**Pandas**](https://pandas.pydata.org/) для анализа данных. Позже в этом курсе мы подробнее поговорим о Pandas и работе с данными в Python.

Давайте вычислим средние значения для возраста, роста и веса:


In [ ]:
df[['Age','Height','Weight']].mean()

Теперь давайте сосредоточимся на росте и вычислим стандартное отклонение и дисперсию:


In [ ]:
print(list(df['Height'])[:20])

In [ ]:
mean = df['Height'].mean()
var = df['Height'].var()
std = df['Height'].std()
print(f"Mean = {mean}\nVariance = {var}\nStandard Deviation = {std}")

В дополнение к среднему значению имеет смысл рассмотреть медиану и квартили. Их можно визуализировать с помощью **ящик с усами**:


In [ ]:
plt.figure(figsize=(10,2))
plt.boxplot(df['Height'].ffill(), vert=False, showmeans=True)
plt.grid(color='gray', linestyle='dotted')
plt.tight_layout()
plt.show()

Мы также можем построить боксплоты для подмножеств нашего датасета, например, сгруппированные по роли игрока.


In [ ]:
df.boxplot(column='Height', by='Role', figsize=(10,8))
plt.xticks(rotation='vertical')
plt.tight_layout()
plt.show()

> **Примечание**: Эта диаграмма предполагает, что в среднем рост первых базменов выше, чем рост вторых базменов. Позже мы узнаем, как можно проверить эту гипотезу более формально и как показать, что наши данные статистически значимы для этого.  

Возраст, рост и вес — это все непрерывные случайные величины. Как вы думаете, какое у них распределение? Хороший способ это выяснить — построить гистограмму значений: 


In [ ]:
df['Weight'].hist(bins=15, figsize=(10,6))
plt.suptitle('Weight distribution of MLB Players')
plt.xlabel('Weight')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

## Нормальное распределение

Давайте создадим искусственную выборку весов, которая следует нормальному распределению с той же средней и дисперсией, что и наши реальные данные:


In [ ]:
generated = np.random.normal(mean, std, 1000)
generated[:20]

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(generated, bins=15)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(np.random.normal(0,1,50000), bins=300)
plt.tight_layout()
plt.show()

Поскольку большинство значений в реальной жизни распределены нормально, не следует использовать генератор случайных чисел с равномерным распределением для генерации выборочных данных. Вот что происходит, если попытаться сгенерировать веса с равномерным распределением (сгенерированным с помощью `np.random.rand`):


In [ ]:
wrong_sample = np.random.rand(1000)*2*std+mean-std
plt.figure(figsize=(10,6))
plt.hist(wrong_sample)
plt.tight_layout()
plt.show()

## Интервалы доверия

Теперь давайте вычислим интервалы доверия для веса и роста бейсболистов. Мы будем использовать код [из этого обсуждения на Stack Overflow](https://stackoverflow.com/questions/15033511/compute-a-confidence-interval-from-sample-data):


In [ ]:
import scipy.stats

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, h

for p in [0.85, 0.9, 0.95]:
    m, h = mean_confidence_interval(df['Weight'].fillna(method='pad'),p)
    print(f"p={p:.2f}, mean = {m:.2f} ± {h:.2f}")

## Проверка гипотез

Давайте рассмотрим различные роли в нашем наборе данных игроков в бейсбол:


In [ ]:
df.groupby('Role').agg({ 'Weight' : 'mean', 'Height' : 'mean', 'Age' : 'count'}).rename(columns={ 'Age' : 'Count'})

Давайте проверим гипотезу о том, что игроки первой базы выше игроков второй базы. Самый простой способ сделать это — проверить доверительные интервалы:


In [ ]:
for p in [0.85,0.9,0.95]:
    m1, h1 = mean_confidence_interval(df.loc[df['Role']=='First_Baseman',['Height']],p)
    m2, h2 = mean_confidence_interval(df.loc[df['Role']=='Second_Baseman',['Height']],p)
    print(f'Conf={p:.2f}, 1st basemen height: {m1-h1[0]:.2f}..{m1+h1[0]:.2f}, 2nd basemen height: {m2-h2[0]:.2f}..{m2+h2[0]:.2f}')

Мы видим, что интервалы не пересекаются.

Статистически более корректным способом проверки гипотезы является использование **t-критерия Стьюдента**:


In [ ]:
from scipy.stats import ttest_ind

tval, pval = ttest_ind(df.loc[df['Role']=='First_Baseman',['Height']], df.loc[df['Role']=='Second_Baseman',['Height']],equal_var=False)
print(f"T-value = {tval[0]:.2f}\nP-value: {pval[0]}")

Два значения, возвращаемые функцией `ttest_ind`, это:
* p-value можно рассматривать как вероятность того, что два распределения имеют одинаковое среднее значение. В нашем случае оно очень низкое, что означает наличие веских доказательств в пользу того, что первичные базмены выше ростом.
* t-value — это промежуточное значение нормализованной разницы средних, которое используется в t-тесте, и оно сравнивается с пороговым значением для данного уровня доверия.


## Моделирование нормального распределения с помощью центральной предельной теоремы

Псевдослучайный генератор в Python предназначен для получения равномерного распределения. Если мы хотим создать генератор для нормального распределения, мы можем использовать центральную предельную теорему. Чтобы получить нормально распределённое значение, мы просто вычислим среднее значение выборки, сгенерированной равномерно.


In [ ]:
def normal_random(sample_size=100):
    sample = [random.uniform(0,1) for _ in range(sample_size) ]
    return sum(sample)/sample_size

sample = [normal_random() for _ in range(100)]
plt.figure(figsize=(10,6))
plt.hist(sample)
plt.tight_layout()
plt.show()

## Корреляция и злая бейсбольная корпорация

Корреляция позволяет нам находить связи между последовательностями данных. В нашем учебном примере представим, что есть злая бейсбольная корпорация, которая платит своим игрокам в зависимости от их роста — чем выше игрок, тем больше он получает денег. Предположим, что базовая зарплата составляет $1000, а дополнительный бонус от $0 до $100 зависит от роста. Мы возьмем реальных игроков из MLB и вычислим их воображаемые зарплаты:


In [ ]:
heights = df['Height'].fillna(method='pad')
salaries = 1000+(heights-heights.min())/(heights.max()-heights.mean())*100
print(list(zip(heights, salaries))[:10])

Давайте теперь вычислим ковариацию и корреляцию этих последовательностей. `np.cov` даст нам так называемую **ковариационную матрицу**, которая является расширением ковариации на несколько переменных. Элемент $M_{ij}$ ковариационной матрицы $M$ — это ковариация между входными переменными $X_i$ и $X_j$, а диагональные значения $M_{ii}$ — дисперсия $X_{i}$. Аналогично, `np.corrcoef` даст нам **корреляционную матрицу**.


In [ ]:
print(f"Covariance matrix:\n{np.cov(heights, salaries)}")
print(f"Covariance = {np.cov(heights, salaries)[0,1]}")
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

Корреляция, равная 1, означает, что существует сильная **линейная связь** между двумя переменными. Мы можем визуально увидеть линейную связь, построив график одного значения относительно другого:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights,salaries)
plt.tight_layout()
plt.show()

Давайте посмотрим, что произойдет, если зависимость не линейная. Предположим, что наша корпорация решила скрыть очевидную линейную зависимость между ростом и зарплатами и ввела некоторую нелинейность в формулу, например, функцию `sin`:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

В этом случае корреляция немного меньше, но она все равно достаточно высокая. Теперь, чтобы сделать связь еще менее очевидной, мы можем добавить некоторую дополнительную случайность, добавив случайную переменную к зарплате. Давайте посмотрим, что получится:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100+np.random.random(size=len(heights))*20-10
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights, salaries)
plt.tight_layout()
plt.show()

> Можете догадаться, почему точки выстраиваются в такие вертикальные линии?

Мы наблюдали корреляцию между искусственно созданной концепцией, такой как зарплата, и наблюдаемой переменной *рост*. Давайте также посмотрим, коррелируют ли две наблюдаемые переменные, такие как рост и вес:


In [ ]:
np.corrcoef(df['Height'].ffill(),df['Weight'])

К сожалению, мы не получили никаких результатов — только странные значения `nan`. Это происходит из-за того, что некоторые значения в нашем ряду неопределены, представлены как `nan`, что приводит к неопределённому результату операции. Глядя на матрицу, мы видим, что проблемным столбцом является `Weight`, так как была вычислена автокорреляция значений `Height`.

> Этот пример демонстрирует важность **подготовки данных** и **очистки**. Без надлежащих данных мы не можем ничего вычислить.

Давайте используем метод `fillna` для заполнения пропущенных значений и вычислим корреляцию: 


In [ ]:
np.corrcoef(df['Height'].fillna(method='pad'), df['Weight'])

Действительно существует корреляция, но не такая сильная, как в нашем искусственном примере. Если посмотреть на точечную диаграмму одного значения относительно другого, связь будет гораздо менее очевидна:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(df['Weight'],df['Height'])
plt.xlabel('Weight')
plt.ylabel('Height')
plt.tight_layout()
plt.show()

## Заключение

В этой тетрадке мы научились выполнять базовые операции с данными для вычисления статистических функций. Теперь мы знаем, как использовать надежный аппарат математики и статистики для проверки гипотез, а также как вычислять доверительные интервалы для произвольных переменных на основе выборки данных.


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Отказ от ответственности**:  
Этот документ был переведен с использованием сервиса машинного перевода [Co-op Translator](https://github.com/Azure/co-op-translator). Хотя мы стремимся к точности, пожалуйста, учтите, что автоматический перевод может содержать ошибки или неточности. Оригинальный документ на его родном языке следует считать авторитетным источником. Для важной информации рекомендуется профессиональный перевод носителем языка. Мы не несем ответственности за любые недоразумения или неверные толкования, возникшие в результате использования данного перевода.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
